In [1]:
!rm wd.sqlite
#Normal Analytics Dependencies#Normal 
import matplotlib
import pandas as pd
import numpy as np
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
from matplotlib import style
import datetime as dt
from sqlalchemy.orm import Session
style.use('seaborn')

#ORM Dependencies
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy import create_engine, Column, Integer, String, Float, inspect, MetaData
from sqlalchemy.ext.declarative import declarative_base

# Declaritive base for table schema creation
Base = declarative_base()

In [2]:
# Bringing in the cleaned CSV's
wd_path = "../Data/CleanedSet.xlsx"

#Reading data
wd_df = pd.read_excel(wd_path, encoding="utf-8")

In [3]:
wd_df.head()

,Type,Rating,Latitude,Longitude,Year,Magnitude,Mass,VEI
0,Volcano,5,41.980,128.080,1000,NaN,NaN,7.0
1,Volcano,5,30.780,130.280,-4350,NaN,NaN,7.0
2,Volcano,5,-8.250,118.000,1815,NaN,NaN,7.0
3,Volcano,5,-30.200,-178.470,-4360,NaN,NaN,6.0
4,Volcano,5,-4.271,152.203,540,NaN,NaN,6.0


In [4]:
wd_df = wd_df[pd.notnull(wd_df['Year'])]

In [5]:
df = wd_df[wd_df.Year.str.contains("Z") == True]
df.head()

,Type,Rating,Latitude,Longitude,Year,Magnitude,Mass,VEI
11599,Earthquake,1,36.344,142.344,520Z,5.8,NaN,NaN
17320,Earthquake,1,8.017,124.075,000Z,5.6,NaN,NaN
19315,Earthquake,1,-32.998,-71.766,530Z,5.6,NaN,NaN


In [6]:
wd_df = wd_df.drop([11599,17320,19315])

In [7]:
wd_df['Year'].astype(float)

0        1000.0
1       -4350.0
2        1815.0
3       -4360.0
4         540.0
5        1280.0
6          60.0
7        1580.0
8         230.0
9         240.0
10        450.0
11       1452.0
12        -50.0
13        710.0
14       1600.0
15        800.0
16        800.0
17        930.0
18       1660.0
19       1912.0
20         50.0
21       -100.0
22      -1860.0
23      -4050.0
24      -4000.0
25      -3580.0
26      -3550.0
27       1883.0
28      -2040.0
29       1902.0
          ...  
69917    2000.0
69918    1998.0
69919    1994.0
69920    1998.0
69921    2000.0
69922    2000.0
69923    1998.0
69924    1996.0
69925    1996.0
69926    1998.0
69927    1999.0
69928    2000.0
69929    2000.0
69930    1996.0
69931    2000.0
69932    2002.0
69933    2002.0
69934    2002.0
69935    2002.0
69936    2002.0
69937    1966.0
69938    1980.0
69939    1968.0
69940    1964.0
69941    2501.0
69942    1931.0
69943    1931.0
69944    1978.0
69945    1979.0
69946    1930.0
Name: Year, Length: 6965

In [8]:
engine = create_engine("sqlite:///wd.sqlite")
conn = engine.connect()

In [9]:
class Measurement(Base):
    __tablename__ = "wd"
    
    index= Column(Integer, primary_key=True)
    Type = Column(String)
    Rating = Column(Integer) 
    Latitude = Column(Float) 
    Longitude = Column(Float)
    Year = Column(Float)
    Magnitude = Column(Float)
    Mass = Column(Float)
    VEI = Column(Integer)

In [10]:
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [11]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [12]:
data_wd = wd_df.to_dict(orient='records')
data_wd[0]

{'Latitude': 41.98,
 'Longitude': 128.08,
 'Magnitude': nan,
 'Mass': nan,
 'Rating': 5,
 'Type': 'Volcano',
 'VEI': 7.0,
 'Year': 1000}

In [13]:
Measure = sqlalchemy.Table('wd', metadata, autoload=True)

In [14]:
conn.execute(Measure.insert(), data_wd)

In [15]:
conn.execute("select * from wd limit 5").fetchall()

[(1, 'Volcano', 5, 41.98, 128.08, 1000.0, None, None, 7),
 (2, 'Volcano', 5, 30.78, 130.28, -4350.0, None, None, 7),
 (3, 'Volcano', 5, -8.25, 118.0, 1815.0, None, None, 7),
 (4, 'Volcano', 5, -30.2, -178.47, -4360.0, None, None, 6),
 (5, 'Volcano', 5, -4.271, 152.203, 540.0, None, None, 6)]